See [github page](https://github.com/LeanManager/NLP-PyTorch/blob/master/Character-Level%20LSTM%20with%20PyTorch.ipynb) or [youtube tutorial](https://www.youtube.com/watch?v=bbvr-2hY4mE&fbclid=IwAR07XZ3NkkK9E5MLBAJOWP3fgWcJXX5_YL7IPZTcBdAw14WL7KPQgJlnHno).

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
# open text file and read in data as `text`
with open('data/text.txt', 'r') as f:
    text = f.read()

In [3]:
# encode the text and map each character to an integer and vice versa

# we create two dictonaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers

chars = tuple(set(text))

int2char = dict(enumerate(chars))

char2int = {ch: ii for ii, ch in int2char.items()}

encoded = np.array([char2int[ch] for ch in text])

In [4]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [5]:
def get_batches(arr, n_seqs, n_steps):
    '''Create a generator that returns batches of size
       n_seqs x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       n_seqs: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    
    batch_size = n_seqs * n_steps
    n_batches = len(arr)//batch_size
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size]
    
    # Reshape into n_seqs rows
    arr = arr.reshape((n_seqs, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        
        # The features
        x = arr[:, n:n+n_steps]
        
        # The targets, shifted by one
        y = np.zeros_like(x)
        
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+n_steps]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [6]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)

In [98]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[76 60  4 68 69  6 31  0  8 82]
 [ 0  4 36  0 37 46 69  0 29 46]
 [62 30 37 58 82 82 12 11  6 35]
 [37  0 26 20 31 30 37 29  0 60]
 [ 0 30 69  0 30 35  9  0 35 30]
 [ 0 72 69  0 28  4 35 82 46 37]
 [60  6 37  0 15 46 36  6  0 43]
 [81  0 66 20 69  0 37 46 28  0]
 [69  0 30 35 37 59 69 58  0 74]
 [ 0 35  4 30 26  0 69 46  0 60]]

y
 [[60  4 68 69  6 31  0  8 82 82]
 [ 4 36  0 37 46 69  0 29 46 30]
 [30 37 58 82 82 12 11  6 35  9]
 [ 0 26 20 31 30 37 29  0 60 30]
 [30 69  0 30 35  9  0 35 30 31]
 [72 69  0 28  4 35 82 46 37 56]
 [ 6 37  0 15 46 36  6  0 43 46]
 [ 0 66 20 69  0 37 46 28  0 35]
 [ 0 30 35 37 59 69 58  0 74 60]
 [35  4 30 26  0 69 46  0 60  6]]


In [90]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_steps=100, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # Creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## Define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## Define a dropout layer
#         self.dropout = nn.Dropout(drop_prob)
        
        ## Define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
        
        # Initialize the weights
        self.init_weights()
        
    def forward(self, x, hc):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hc`. '''
        
        ## Get x, and the new hidden state (h, c) from the lstm
        x, (h, c) = self.lstm(x, hc)
        
        ## Ppass x through the dropout layer
#         x = self.dropout(x)
        
        # Stack up LSTM outputs using view
        x = x.contiguous().view(-1, self.n_hidden)
        
        ## Put x through the fully-connected layer
        x = self.fc(x)
        
        # Return x and the hidden state (h, c)
        return x, (h, c)
    
    def predict(self, char, h=None, cuda=False, top_k=None):
        ''' Given a character, predict the next character.
        
            Returns the predicted character and the hidden state.
        '''
        if cuda:
            self.cuda()
        else:
            self.cpu()
        
        if h is None:
            h = self.init_hidden(1)
        
        x = np.array([[self.char2int[char]]])
        x = one_hot_encode(x, len(self.chars))
        
        inputs = torch.from_numpy(x)
        
        if cuda:
            inputs = inputs.cuda()
        
        h = tuple([each.data for each in h])
        out, h = self.forward(inputs, h)

        p = F.softmax(out, dim=1).data
        
        if cuda:
            p = p.cpu()
        
        if top_k is None:
            top_ch = np.arange(len(self.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        p = p.numpy().squeeze()
        
        char = np.random.choice(top_ch, p=p/p.sum())
            
        return self.int2char[char], h
    
    def init_weights(self):
        ''' Initialize weights for fully connected layer '''
        initrange = 0.1
        
        # Set bias tensor to all zeros
        self.fc.bias.data.fill_(0)
        # FC weights as random uniform
        self.fc.weight.data.uniform_(-1, 1)
        
    def init_hidden(self, n_seqs):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x n_seqs x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        return (weight.new(self.n_layers, n_seqs, self.n_hidden).zero_(),
                weight.new(self.n_layers, n_seqs, self.n_hidden).zero_())

In [91]:
def train(net, data, epochs=10, n_seqs=10, n_steps=50, lr=0.001, clip=5, val_frac=0.1, cuda=False, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        n_seqs: Number of mini-sequences per mini-batch, aka batch size
        n_steps: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        cuda: Train with CUDA on a GPU
        print_every: Number of steps for printing training and validation loss
    
    '''
    
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if cuda:
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    
    for e in range(epochs):
        
        h = net.init_hidden(n_seqs)
        
        for x, y in get_batches(data, n_seqs, n_steps):
            
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            net.zero_grad()
                        
            output, h = net.forward(inputs, h)
            
            loss = criterion(output, targets.view(n_seqs*n_steps))

            loss.backward()
            
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)

            opt.step()
            
            if counter % print_every == 0:
                
                # Get validation loss
                val_h = net.init_hidden(n_seqs)
                val_losses = []
                
                for x, y in get_batches(val_data, n_seqs, n_steps):
                    
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if cuda:
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net.forward(inputs, val_h)
                    val_loss = criterion(output, targets.view(n_seqs*n_steps))
                
                    val_losses.append(val_loss.item())
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [92]:
if 'net' in locals():
    del net

In [93]:
# Initialize and print the network
net = CharRNN(chars, n_hidden=512, n_layers=2)

print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


In [94]:
n_seqs, n_steps = 128, 100

In [95]:
train(net, encoded, epochs=25, n_seqs=n_seqs, n_steps=n_steps, lr=0.001, print_every=10)

Epoch: 1/25... Step: 10... Loss: 3.1860... Val Loss: 3.1672
Epoch: 1/25... Step: 20... Loss: 3.0272... Val Loss: 3.0505
Epoch: 1/25... Step: 30... Loss: 2.9378... Val Loss: 2.9336
Epoch: 1/25... Step: 40... Loss: 2.7417... Val Loss: 2.7610
Epoch: 1/25... Step: 50... Loss: 2.6184... Val Loss: 2.5988
Epoch: 1/25... Step: 60... Loss: 2.4755... Val Loss: 2.5083
Epoch: 1/25... Step: 70... Loss: 2.4179... Val Loss: 2.4497
Epoch: 1/25... Step: 80... Loss: 2.3532... Val Loss: 2.3924
Epoch: 1/25... Step: 90... Loss: 2.3456... Val Loss: 2.3543
Epoch: 1/25... Step: 100... Loss: 2.2831... Val Loss: 2.3178
Epoch: 1/25... Step: 110... Loss: 2.2383... Val Loss: 2.2912
Epoch: 1/25... Step: 120... Loss: 2.1711... Val Loss: 2.2651
Epoch: 1/25... Step: 130... Loss: 2.2074... Val Loss: 2.2312


KeyboardInterrupt: 

In [99]:
def sample(net, size, prime='The', top_k=None, cuda=False):
        
    if cuda:
        net.cuda()
    else:
        net.cpu()

    net.eval()
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    
    h = net.init_hidden(1)
    
    for ch in prime:
        char, h = net.predict(ch, h, cuda=cuda, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        
        char, h = net.predict(chars[-1], h, cuda=cuda, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [101]:
print(sample(net, 2000, prime='Anna', top_k=5))

Annat the
seling to steaned, to he tored...
"The wathe hererathin, arder oned he morsered
ing taing hit oully.
"Whell oug har walid.

"Whe sare her sad thard thoug this bled ha dor hiss, atd one to mased tereder hers of the sar ines and took and ha tore that he howshing he pat the hing te hes to hered had wort he the thas
been he wored, anded to to ke fithe hord than wat an tale fas thas buting ante se to to the pitith the to ber hate, and he shad dore tit on the perestere, sain he sale, the wor ine hit he who gouth that at toong it he her and withe her tarde fore too the sarid tome alle fer ond he mald to the wald and of of ofly he sher hersted the frime thang of tha bat and avion tore ano sored, at the har
seled and onge the he sardede till thanderinging at
ha ther thor alladingert of houll an of allen that of thas he hure tor tore tor inted of the hirg of the the tareres fomint, ald ande saly wist ores im ofere thought he toute th tering, the said hime.. SWerente farentines.

"Wast,